[Reference](https://medium.com/@ilaslanduzgun/how-to-serve-machine-learning-model-using-fastapi-mlflow-minio-mysql-8b99a6c76989)

In [1]:
# Determine the Urls
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5001/'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000/'

# Set MLflow experiment
experiment_name = "Deploy Model using FastAPI-MLflow-MINIO"
mlflow.set_experiment(experiment_name)

registered_model_name="RFElectricityPricePrediction"

# Train model and register mlflow
with mlflow.start_run(run_name="with-reg-rf-sklearn") as run:
    estimator = RandomForestRegressor(n_estimators=number_of_trees)
    estimator.fit(X_train, y_train)

    y_pred = estimator.predict(X_test)

    (rmse, mae, r2) = eval_metrics(y_test, y_pred)

    print(f"Random Forest model number of trees: {number_of_trees}")
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("n_estimators", number_of_trees)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        # Register the model
        mlflow.sklearn.log_model(estimator, "model", registered_model_name=registered_model_name)
    else:
        mlflow.sklearn.log_model(estimator, "model")

from typing import Optional
from datetime import datetime
from sqlmodel import SQLModel, Field

class PricePredictions(SQLModel,table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    Day: int
    Month: int
    ForecastWindProduction: float
    SystemLoadEA: float
    SMPEA: float
    ORKTemperature: float
    ORKWindspeed: float
    CO2Intensity: float
    ActualWindProduction: float
    SystemLoadEP2: float
    prediction: float
    prediction_time: datetime = Field(default_factory=datetime.utcnow, nullable=False)
    client_ip: str

class Price(SQLModel):
    Day: int
    Month: int
    ForecastWindProduction: float
    SystemLoadEA: float
    SMPEA: float
    ORKTemperature: float
    ORKWindspeed: float
    CO2Intensity: float
    ActualWindProduction: float
    SystemLoadEP2: float

    class Config:
        schema_extra = {
            "example": {
                "Day": 10,
                "Month": 12,
                "ForecastWindProduction": 54.10,
                "SystemLoadEA": 4241.05,
                "SMPEA":49.56,
                "ORKTemperature":9.0,
                "ORKWindspeed":14.8,
                "CO2Intensity":491.32,
                "ActualWindProduction":54.0,
                "SystemLoadEP2":4426.84

            }
        }

from mlflow.sklearn import load_model

# Learn, decide and get model from mlflow model registry
model_name = "RFElectricityPricePrediction"
model_version = 1
model = load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

# Electirical Price Prediction endpoint
@app.post("/prediction/priceprediction")
async def predictPrice(request: Price, fastapi_req: Request,  db: Session = Depends(get_db)):
    prediction = makePrediction(model, request.dict())
    db_insert_record = insertPrice(request=request.dict(), prediction=prediction,
                                          client_ip=fastapi_req.client.host,
                                          db=db)
    return {"prediction": prediction, "db_record": db_insert_record}

import os
from dotenv import load_dotenv
from sqlmodel import create_engine, SQLModel, Session

load_dotenv()  # take environment variables from .env.
SQLALCHEMY_DATABASE_URL = os.getenv('SQLALCHEMY_DATABASE_URL')

# Create engine
engine = create_engine(SQLALCHEMY_DATABASE_URL, echo=True)

# Create table
def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

# Connet to database
def get_db():
    db = Session(engine)
    try:
        yield db
    finally:
        db.close()

from database import engine, get_db, create_db_and_tables
# Creates all the tables defined in models module
create_db_and_tables()

# This is a not part of the flow code, this is quotation from endpoint part of FastAPI code method
# This is about demostrations.
async def predictPrice(request: Price, fastapi_req: Request,  db: Session = Depends(get_db)):